In [7]:
import cv2
import numpy as np
from PIL import Image
import zipfile
import os
import matplotlib
import dlib

In [2]:
face_detector = dlib.get_frontal_face_detector()
points_detector = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [4]:
image = cv2.imread('0.jpg')
face_detection = face_detector(image, 1)
for face in face_detection:
  points = points_detector(image, face)
  for point in points.parts():
    cv2.circle(image, (point.x, point.y), 2, (0,255,0), 1)

  print(len(points.parts()), points.parts())

  l, t, r, b = face.left(), face.top(), face.right(), face.bottom()
  cv2.rectangle(image, (l, t), (r, b), (0,255,255), 2)
cv2.imshow("AI",image)
cv2.waitKey(5000)
cv2.destroyAllWindows()

68 points[(365, 369), (366, 394), (370, 418), (375, 441), (383, 464), (396, 485), (413, 505), (431, 519), (451, 525), (474, 522), (496, 510), (516, 493), (532, 474), (543, 450), (549, 425), (554, 398), (557, 374), (377, 344), (387, 330), (403, 325), (421, 327), (436, 335), (472, 335), (490, 328), (509, 328), (528, 334), (538, 350), (452, 355), (451, 373), (450, 390), (448, 409), (430, 418), (440, 422), (450, 425), (460, 422), (470, 419), (396, 359), (406, 351), (419, 352), (430, 362), (418, 364), (405, 363), (479, 364), (490, 355), (504, 355), (515, 362), (505, 367), (491, 367), (413, 446), (425, 440), (439, 439), (449, 441), (460, 440), (476, 443), (493, 449), (477, 471), (461, 478), (449, 479), (438, 476), (424, 465), (419, 447), (439, 445), (449, 447), (460, 446), (488, 450), (461, 464), (449, 465), (439, 463)]


In [4]:
path = 'archive.zip'
zip_object = zipfile.ZipFile(file = path, mode = 'r')
zip_object.extractall('./')
zip_object.close()

## Detecting facial descriptors

In [5]:
face_descriptor_extractor = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')

In [7]:
test_image = '9.jpg'
image = Image.open(test_image).convert('L')
image_np = np.array(image, 'uint8')
image_np = cv2.cvtColor(image_np, cv2.COLOR_GRAY2RGB)

face_detection = face_detector(image_np, 1)
for face in face_detection:
  l, t, r, b = face.left(), face.top(), face.right(), face.bottom()
  cv2.rectangle(image_np, (l, t), (r, b), (255,0,255), 2)

  points = points_detector(image_np, face)
  for point in points.parts():
    cv2.circle(image_np, (point.x, point.y), 2, (0,255,0), 1)

  face_descriptor = face_descriptor_extractor.compute_face_descriptor(image_np, points)
  
  face_descriptor = [f for f in face_descriptor]
  
  face_descriptor = np.asarray(face_descriptor, dtype = np.float64)
 
  face_descriptor = face_descriptor[np.newaxis, :]
  

cv2.imshow("AI",image_np)
cv2.waitKey(2000)
cv2.destroyAllWindows()

In [8]:
def extract_descriptor(face, image_np, face_descriptors):
  l, t, r, b = face.left(), face.top(), face.right(), face.bottom()
  cv2.rectangle(image_np, (l, t), (r, b), (0, 255, 0), 2)

  points = points_detector(image_np, face)
  for point in points.parts():
    cv2.circle(image_np, (point.x, point.y), 2, (0, 255, 0), 1)

  face_descriptor = face_descriptor_extractor.compute_face_descriptor(image_np, points)
  face_descriptor = [f for f in face_descriptor]
  face_descriptor = np.asarray(face_descriptor, dtype=np.float64)
  face_descriptor = face_descriptor[np.newaxis, :]

  if face_descriptors is None:
    face_descriptors = face_descriptor
  else:
    face_descriptors = np.concatenate((face_descriptors, face_descriptor), axis = 0)

  return image_np, face_descriptors

## Loading the training set

In [9]:
def load_training(path_dataset):
  index = {}
  idx = 0
  face_descriptors = None

  paths = [os.path.join(path_dataset, f) for f in os.listdir(path_dataset)]

  for path in paths:
    #print(path)
    image = Image.open(path).convert('L')
    image_np = np.array(image, 'uint8')
    image_np = cv2.cvtColor(image_np, cv2.COLOR_GRAY2RGB)

    face_detection = face_detector(image_np, 1)
    for face in face_detection:
      image_np, face_descriptors = extract_descriptor(face, image_np, face_descriptors)
      index[idx] = path
      idx += 1
    #cv2.imshow("TEST",image_np)
    #cv2.waitKey(2000)
    #cv2.destroyAllWindows()
  return face_descriptors, index

In [10]:
face_descriptors, index = load_training('archive')

## Recognizing Faces

In [15]:
def predict_images(path_dataset, face_descriptors, index, threshold=0.5):
  predictions = []
   
  expected_outputs = []
  paths = [os.path.join(path_dataset, f) for f in os.listdir(path_dataset)]
  for path in paths:
    image = Image.open(path).convert('RGB')
    image_np = np.array(image, 'uint8')
    (h, w) = image_np.shape[:2]
    face_detection = face_detector(image_np, 1)
    for face in face_detection:
      points = points_detector(image_np, face)
      face_descriptor = face_descriptor_extractor.compute_face_descriptor(image_np, points)
      face_descriptor = [f for f in face_descriptor]
      face_descriptor = np.asarray(face_descriptor, dtype=np.float64)
      face_descriptor = face_descriptor[np.newaxis, :]

      distances = np.linalg.norm(face_descriptor - face_descriptors, axis = 1)
      min_index = np.argmin(distances)
      min_distance = distances[min_index]
      if min_distance <= threshold:
        name_pred = os.path.split(index[min_index])[1].split('.')[0].replace('subject', '')
      else:
        name_pred = -1 # not identified

      name_real = os.path.split(path)[1].split('.')[0].replace('subject', '')

      predictions.append(name_pred)
      expected_outputs.append(name_real)

      cv2.putText(image_np, 'Pred: ' + str(name_pred), (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,255))
      cv2.putText(image_np, 'Exp: ' + str(name_real), (10,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0))
      cv2.putText(image_np, str(min_distance), (10,h - 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.6, (0,0,255))
      
        
    cv2.imshow("test",image_np)
    cv2.waitKey(3000)
    cv2.destroyAllWindows()

  predictions = np.array(predictions)
  expected_outputs = np.array(expected_outputs)

  return predictions, expected_outputs

In [16]:
predictions, expected_outputs = predict_images('TEST_MODEL', face_descriptors, index)

## Saving Descriptors for future use ( Python Pickle)


In [22]:
import pickle
np.save('face_descriptors.npy', face_descriptors)
with open('index_faces.pickle', 'wb') as f:
  pickle.dump(index, f)

In [23]:
index = np.load('index_faces.pickle', allow_pickle = True)
face_descriptors = np.load('face_descriptors.npy')

## Display recognition above bounding box

In [12]:
import face_recognition

In [11]:
paths = [os.path.join('archive', f) for f in os.listdir('archive') if f.endswith('.jpg')]

In [9]:
def recognize_faces(image, list_encodings, list_names, tolerance = 0.6):
  img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  face_locations = face_recognition.face_locations(img_rgb)
  face_encodings = face_recognition.face_encodings(img_rgb, face_locations)
  face_names = []
  conf_values = []
  for encoding in face_encodings:
    matches = face_recognition.compare_faces(list_encodings, encoding, tolerance = tolerance)
    name = 'Not identified'
    face_distances = face_recognition.face_distance(list_encodings, encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
      name = list_names[best_match_index]
    face_names.append(name)
    conf_values.append(face_distances[best_match_index])
  face_locations = np.array(face_locations)
  return face_locations.astype(int), face_names, conf_values

In [10]:
def get_encodings(paths):
  print('{} images found'.format(len(paths)))
  list_encodings = []
  list_names = []
  for img_path in paths:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    basename = os.path.basename(img_path)
    #print(basename)
    (name, ext) = os.path.splitext(basename)
    #print(name, ext)
    face_roi = face_recognition.face_locations(img, model = 'cnn') # hog
    face_encoding = face_recognition.face_encodings(img, face_roi)[0]
    if len(face_encoding) > 0:
      list_encodings.append(face_encoding)
      list_names.append(name)
    else:
      print('Could not detect the face from image {}'.format(img_path))
  return list_encodings, list_names

In [21]:
list_encodings, list_names = get_encodings(paths)

10 images found


In [29]:
#with open('listEncodings.pickle','wb') as f:
    #pickle.dump(list_encodings,f)

with open('listNames.pickle','wb') as f:
    pickle.dump(list_names,f)

In [2]:
import numpy as np

In [3]:
list_encodings = np.load('listEncodings.pickle', allow_pickle = True)
list_names = np.load('listNames.pickle', allow_pickle = True)

In [4]:
import imutils

In [5]:
def show_recognition(test_image, list_encodings, list_names, max_width=700, tolerance=0.6):
  face_locations, face_names, conf_values = recognize_faces(test_image, list_encodings, list_names, tolerance)

  for face_loc, name, conf in zip(face_locations, face_names, conf_values):
    y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

    cv2.putText(test_image, name,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0), 1)
    cv2.rectangle(test_image, (x1, y1), (x2, y2), (0,255,0), 3)
    print(conf)

  if (test_image.shape[1] > max_width):
    test_image = imutils.resize(test_image, width=max_width)
  cv2.imshow("AI",test_image)
  cv2.waitKey(10000)  
  cv2.destroyAllWindows()

In [13]:
test_image = cv2.imread('0.jpg')
show_recognition(test_image, list_encodings, list_names)

0.7948475879377213
